In [83]:
import pandas as pd

res = pd.read_csv('https://raw.githubusercontent.com/petebrown/data-updater/refs/heads/main/data/results.csv', parse_dates=['game_date'])
stats = pd.read_csv('data/match_stats.csv', parse_dates=['game_date']).rename(columns={'possessionPercentage': 'possession'})
goals = pd.read_csv('data/sameday_scorers.csv', parse_dates=['game_date'])

In [84]:
lge_dates = res.query('game_type=="League"')[['game_date']]

In [85]:
opp_goals = goals.merge(res[['game_date', 'opposition']], left_on=['game_date', 'team_name'], right_on=['game_date', 'opposition'])
tr_goals = goals.query('team_name == "Tranmere Rovers"')
match_goals = pd.concat([tr_goals, opp_goals], ignore_index=True).query('game_date.isin(@lge_dates.game_date)')
match_goals['goal_min'] = match_goals['goal_min'].str.replace(' ET', '')
match_goals['goal_min'] = match_goals['goal_min'].astype(int)
match_goals = match_goals.sort_values(['game_date', 'goal_min'])
match_goals = match_goals.merge(res[['game_date', 'score', 'outcome', 'manager']], on='game_date').drop(columns=['bbc_player_id', 'goal_min_inj'])

match_goals.tail(20)

,game_date,team_name,player_name,goal_min,goal_type,opposition,score,outcome,manager
449,2024-04-20,Tranmere Rovers,H. Saunders,35,Goal,NaN,3-2,W,Nigel Adkins
450,2024-04-20,Tranmere Rovers,R. Apter,45,Goal,NaN,3-2,W,Nigel Adkins
451,2024-04-20,AFC Wimbledon,J. Kelly,56,Goal,AFC Wimbledon,3-2,W,Nigel Adkins
452,2024-04-20,Tranmere Rovers,C. Jennings,71,Goal,NaN,3-2,W,Nigel Adkins
453,2024-04-20,AFC Wimbledon,J. Kelly,90,Goal,AFC Wimbledon,3-2,W,Nigel Adkins
454,2024-04-27,Tranmere Rovers,L. Norris,2,Goal,NaN,1-4,L,Nigel Adkins
455,2024-04-27,Accrington Stanley,L. Shipley,4,Goal,Accrington Stanley,1-4,L,Nigel Adkins
456,2024-04-27,Accrington Stanley,J. Nolan,51,Penalty,Accrington Stanley,1-4,L,Nigel Adkins
457,2024-04-27,Accrington Stanley,A. Henderson,53,Goal,Accrington Stanley,1-4,L,Nigel Adkins
458,2024-04-27,Accrington Stanley,J. Nolan,82,Penalty,Accrington Stanley,1-4,L,Nigel Adkins


In [86]:
outcomes = res.query('game_date >= @stats.game_date.min()')[['game_date', 'outcome', 'venue', 'opposition', 'score', 'competition', 'game_type', 'league_tier', 'manager']]

In [87]:
poss = stats.query('team_name == "Tranmere Rovers"')[['game_date', 'possession']]

poss

,game_date,possession
0,2020-01-01,42.1
3,2020-01-04,44.0
4,2020-01-07,43.1
7,2020-01-11,51.6
8,2020-01-18,44.2
...,...,...
475,2024-08-31,38.8
477,2024-09-14,63.2
479,2024-09-21,64.3
480,2024-09-27,53.3


In [88]:
poss['poss_bin'] = pd.cut(poss['possession'], bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], labels=['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100'])

poss['poss_over50'] = poss['possession'] >= 50

poss

,game_date,possession,poss_bin,poss_over50
0,2020-01-01,42.1,40-50,False
3,2020-01-04,44.0,40-50,False
4,2020-01-07,43.1,40-50,False
7,2020-01-11,51.6,50-60,True
8,2020-01-18,44.2,40-50,False
...,...,...,...,...
475,2024-08-31,38.8,30-40,False
477,2024-09-14,63.2,60-70,True
479,2024-09-21,64.3,60-70,True
480,2024-09-27,53.3,50-60,True


In [89]:
df = poss.merge(outcomes, on='game_date', how='outer')

df

,game_date,possession,poss_bin,poss_over50,outcome,venue,opposition,score,competition,game_type,league_tier,manager
0,2020-01-01,42.1,40-50,False,L,H,Coventry City,1-4,League One,League,3.0,Micky Mellon
1,2020-01-04,44.0,40-50,False,D,A,Watford,3-3,FA Cup,Cup,NaN,Micky Mellon
2,2020-01-07,43.1,40-50,False,L,H,Leicester City,1-2,Football League Trophy,Cup,NaN,Micky Mellon
3,2020-01-11,51.6,50-60,True,D,A,Southend United,0-0,League One,League,3.0,Micky Mellon
4,2020-01-18,44.2,40-50,False,L,H,Ipswich Town,1-2,League One,League,3.0,Micky Mellon
...,...,...,...,...,...,...,...,...,...,...,...,...
240,2024-08-31,38.8,30-40,False,W,A,Carlisle United,2-1,League Two,League,4.0,Nigel Adkins
241,2024-09-14,63.2,60-70,True,L,A,Gillingham,0-3,League Two,League,4.0,Nigel Adkins
242,2024-09-21,64.3,60-70,True,L,A,Colchester United,0-3,League Two,League,4.0,Nigel Adkins
243,2024-09-27,53.3,50-60,True,D,H,Salford City,0-0,League Two,League,4.0,Nigel Adkins


In [90]:
lge = df.query('game_type=="League"').copy()

lge['pts'] = lge['outcome'].apply(lambda x: 3 if x == 'W' else 1 if x == 'D' else 0)

lge

,game_date,possession,poss_bin,poss_over50,outcome,venue,opposition,score,competition,game_type,league_tier,manager,pts
0,2020-01-01,42.1,40-50,False,L,H,Coventry City,1-4,League One,League,3.0,Micky Mellon,0
3,2020-01-11,51.6,50-60,True,D,A,Southend United,0-0,League One,League,3.0,Micky Mellon,1
4,2020-01-18,44.2,40-50,False,L,H,Ipswich Town,1-2,League One,League,3.0,Micky Mellon,0
7,2020-01-29,49.9,40-50,False,L,H,Sunderland,0-1,League One,League,3.0,Micky Mellon,0
8,2020-02-01,43.1,40-50,False,L,A,Bolton Wanderers,0-2,League One,League,3.0,Micky Mellon,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,2024-08-31,38.8,30-40,False,W,A,Carlisle United,2-1,League Two,League,4.0,Nigel Adkins,3
241,2024-09-14,63.2,60-70,True,L,A,Gillingham,0-3,League Two,League,4.0,Nigel Adkins,0
242,2024-09-21,64.3,60-70,True,L,A,Colchester United,0-3,League Two,League,4.0,Nigel Adkins,0
243,2024-09-27,53.3,50-60,True,D,H,Salford City,0-0,League Two,League,4.0,Nigel Adkins,1


In [91]:
l2 = lge.query('game_date >= "2020-08-01"')

l2

,game_date,possession,poss_bin,poss_over50,outcome,venue,opposition,score,competition,game_type,league_tier,manager,pts
18,2020-09-12,49.2,40-50,False,D,A,Mansfield Town,0-0,League Two,League,4.0,Michael Jackson,1
19,2020-09-19,44.4,40-50,False,L,H,Cheltenham Town,0-3,League Two,League,4.0,Michael Jackson,0
20,2020-09-26,57.5,50-60,True,D,A,Cambridge United,0-0,League Two,League,4.0,Michael Jackson,1
22,2020-10-03,47.0,40-50,False,W,H,Scunthorpe United,2-0,League Two,League,4.0,Michael Jackson,3
23,2020-10-10,48.9,40-50,False,D,A,Salford City,2-2,League Two,League,4.0,Michael Jackson,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,2024-08-31,38.8,30-40,False,W,A,Carlisle United,2-1,League Two,League,4.0,Nigel Adkins,3
241,2024-09-14,63.2,60-70,True,L,A,Gillingham,0-3,League Two,League,4.0,Nigel Adkins,0
242,2024-09-21,64.3,60-70,True,L,A,Colchester United,0-3,League Two,League,4.0,Nigel Adkins,0
243,2024-09-27,53.3,50-60,True,D,H,Salford City,0-0,League Two,League,4.0,Nigel Adkins,1


In [92]:
l2.groupby(['outcome'])['possession'].median().to_frame().reset_index()

,outcome,possession
0,D,51.45
1,L,53.05
2,W,47.20


In [93]:
l2.groupby(['manager', 'outcome'])['possession'].median().to_frame().reset_index()

,manager,outcome,possession
0,Ian Dawes,D,55.75
1,Ian Dawes,L,53.00
2,Ian Dawes,W,49.85
3,Keith Hill,D,56.40
4,Keith Hill,L,61.45
5,Keith Hill,W,52.10
6,Michael Jackson,D,49.20
7,Michael Jackson,L,45.50
8,Michael Jackson,W,46.90
9,Micky Mellon,D,47.80


In [94]:
l2.query('manager=="Nigel Adkins" & outcome=="W"')['poss_bin'].value_counts(normalize=False).sort_index().to_frame().T

poss_bin,0-10,10-20,20-30,30-40,40-50,50-60,60-70,70-80,80-90,90-100
count,0,0,0,5,9,4,0,0,0,0


In [95]:
l2.query('manager=="Nigel Adkins" & outcome=="W"')['poss_bin'].value_counts(normalize=True).sort_index().to_frame().T

poss_bin,0-10,10-20,20-30,30-40,40-50,50-60,60-70,70-80,80-90,90-100
proportion,0.0,0.0,0.0,0.277778,0.5,0.222222,0.0,0.0,0.0,0.0


# Wins

In [96]:
l2.query("outcome=='W'").groupby('manager')['poss_bin'].value_counts(normalize=True).sort_index().to_frame().reset_index() \
    .pivot(index='manager', columns='poss_bin', values='proportion')

poss_bin,0-10,10-20,20-30,30-40,40-50,50-60,60-70,70-80,80-90,90-100
manager,,,,,,,,,,
Ian Dawes,0.0,0.0,0.000000,0.166667,0.333333,0.333333,0.166667,0.000000,0.0,0.0
Keith Hill,0.0,0.0,0.000000,0.000000,0.466667,0.333333,0.133333,0.066667,0.0,0.0
Michael Jackson,0.0,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0
Micky Mellon,0.0,0.0,0.029412,0.147059,0.470588,0.294118,0.058824,0.000000,0.0,0.0
Nigel Adkins,0.0,0.0,0.000000,0.277778,0.500000,0.222222,0.000000,0.000000,0.0,0.0


In [97]:
l2.query("outcome=='W'").groupby('manager')['poss_bin'].value_counts(normalize=False).sort_index().to_frame().reset_index() \
    .pivot(index='manager', columns='poss_bin', values='count')

poss_bin,0-10,10-20,20-30,30-40,40-50,50-60,60-70,70-80,80-90,90-100
manager,,,,,,,,,,
Ian Dawes,0,0,0,1,2,2,1,0,0,0
Keith Hill,0,0,0,0,7,5,2,1,0,0
Michael Jackson,0,0,0,0,2,0,0,0,0,0
Micky Mellon,0,0,1,5,16,10,2,0,0,0
Nigel Adkins,0,0,0,5,9,4,0,0,0,0


In [98]:
l2.query("outcome=='W'").groupby('manager')['poss_over50'].value_counts(normalize=True).reset_index()

,manager,poss_over50,proportion
0,Ian Dawes,False,0.500000
1,Ian Dawes,True,0.500000
2,Keith Hill,True,0.533333
3,Keith Hill,False,0.466667
4,Michael Jackson,False,1.000000
5,Micky Mellon,False,0.647059
6,Micky Mellon,True,0.352941
7,Nigel Adkins,False,0.777778
8,Nigel Adkins,True,0.222222


In [99]:
l2.query("outcome=='W'").groupby('manager')['poss_over50'].value_counts(normalize=False).reset_index()

,manager,poss_over50,count
0,Ian Dawes,False,3
1,Ian Dawes,True,3
2,Keith Hill,True,8
3,Keith Hill,False,7
4,Michael Jackson,False,2
5,Micky Mellon,False,22
6,Micky Mellon,True,12
7,Nigel Adkins,False,14
8,Nigel Adkins,True,4


# Losses

In [100]:
l2.query("outcome=='L'").groupby('manager')['poss_bin'].value_counts(normalize=False).sort_index().to_frame().reset_index() \
    .pivot(index='manager', columns='poss_bin', values='count')

poss_bin,0-10,10-20,20-30,30-40,40-50,50-60,60-70,70-80,80-90,90-100
manager,,,,,,,,,,
Ian Dawes,0,0,0,1,3,2,3,0,0,0
Keith Hill,0,0,0,0,0,3,5,0,0,0
Michael Jackson,0,0,0,0,3,1,1,0,0,0
Micky Mellon,0,0,0,1,9,9,7,1,0,0
Nigel Adkins,0,0,2,1,6,7,3,0,0,0


In [101]:
l2.query("outcome=='L'").groupby('manager')['poss_bin'].value_counts(normalize=True).sort_index().to_frame().reset_index() \
    .pivot(index='manager', columns='poss_bin', values='proportion')

poss_bin,0-10,10-20,20-30,30-40,40-50,50-60,60-70,70-80,80-90,90-100
manager,,,,,,,,,,
Ian Dawes,0.0,0.0,0.000000,0.111111,0.333333,0.222222,0.333333,0.000000,0.0,0.0
Keith Hill,0.0,0.0,0.000000,0.000000,0.000000,0.375000,0.625000,0.000000,0.0,0.0
Michael Jackson,0.0,0.0,0.000000,0.000000,0.600000,0.200000,0.200000,0.000000,0.0,0.0
Micky Mellon,0.0,0.0,0.000000,0.037037,0.333333,0.333333,0.259259,0.037037,0.0,0.0
Nigel Adkins,0.0,0.0,0.105263,0.052632,0.315789,0.368421,0.157895,0.000000,0.0,0.0


# Results

In [102]:
man_counts = l2.query("outcome=='W'").groupby(['manager', 'poss_bin'], observed=True)['score'].value_counts().sort_index().to_frame().reset_index() \
    .query('count>0')

man_counts['as_%'] = man_counts.groupby('manager')['count'].transform(lambda x: round(x / x.sum() * 100, 1))

man_counts

,manager,poss_bin,score,count,as_%
0,Ian Dawes,30-40,1-0,1,16.7
1,Ian Dawes,40-50,2-0,1,16.7
2,Ian Dawes,40-50,3-0,1,16.7
3,Ian Dawes,50-60,1-0,1,16.7
4,Ian Dawes,50-60,5-0,1,16.7
5,Ian Dawes,60-70,4-3,1,16.7
6,Keith Hill,40-50,1-0,3,20.0
7,Keith Hill,40-50,2-1,2,13.3
8,Keith Hill,40-50,3-0,1,6.7
9,Keith Hill,40-50,3-1,1,6.7


In [103]:
l2.query('manager=="Nigel Adkins" & outcome=="W" & possession>=50')

,game_date,possession,poss_bin,poss_over50,outcome,venue,opposition,score,competition,game_type,league_tier,manager,pts
193,2023-09-23,53.6,50-60,True,W,H,Accrington Stanley,2-0,League Two,League,4.0,Nigel Adkins,3
208,2023-12-09,59.3,50-60,True,W,H,Newport County,2-1,League Two,League,4.0,Nigel Adkins,3
210,2023-12-23,51.9,50-60,True,W,H,Swindon Town,2-1,League Two,League,4.0,Nigel Adkins,3
232,2024-04-20,51.2,50-60,True,W,H,AFC Wimbledon,3-2,League Two,League,4.0,Nigel Adkins,3


In [104]:
l2.query('manager=="Nigel Adkins" & outcome=="W" & possession<50').sort_values('possession')

,game_date,possession,poss_bin,poss_over50,outcome,venue,opposition,score,competition,game_type,league_tier,manager,pts
213,2024-01-01,33.0,30-40,False,W,H,Notts County,4-2,League Two,League,4.0,Nigel Adkins,3
206,2023-11-25,33.6,30-40,False,W,H,Gillingham,3-1,League Two,League,4.0,Nigel Adkins,3
240,2024-08-31,38.8,30-40,False,W,A,Carlisle United,2-1,League Two,League,4.0,Nigel Adkins,3
238,2024-08-24,39.2,30-40,False,W,H,Walsall,1-0,League Two,League,4.0,Nigel Adkins,3
220,2024-02-17,39.9,30-40,False,W,H,Stockport County,4-0,League Two,League,4.0,Nigel Adkins,3
212,2023-12-29,40.7,40-50,False,W,A,Harrogate Town,2-0,League Two,League,4.0,Nigel Adkins,3
195,2023-10-03,43.4,40-50,False,W,H,Bradford City,2-1,League Two,League,4.0,Nigel Adkins,3
203,2023-11-11,44.3,40-50,False,W,H,Forest Green Rovers,3-0,League Two,League,4.0,Nigel Adkins,3
226,2024-03-16,44.4,40-50,False,W,A,Wrexham,1-0,League Two,League,4.0,Nigel Adkins,3
216,2024-01-27,44.7,40-50,False,W,A,Grimsby Town,2-1,League Two,League,4.0,Nigel Adkins,3


In [105]:
first_goals = match_goals.groupby(['game_date']).first().reset_index()

first_goals['for_against'] = first_goals['team_name'].apply(lambda x: 'for' if x == 'Tranmere Rovers' else 'against')

first_goals = first_goals.merge(poss, on='game_date', how='left')

first_goals.query('manager=="Nigel Adkins" & for_against=="against"')

,game_date,team_name,player_name,goal_min,goal_type,opposition,score,outcome,manager,for_against,possession,poss_bin,poss_over50
147,2023-10-14,Crewe Alexandra,M. Demetriou,11,Goal,Crewe Alexandra,0-2,L,Nigel Adkins,against,44.0,40-50,False
148,2023-10-20,Doncaster Rovers,H. Biggins,27,Goal,Doncaster Rovers,1-2,L,Nigel Adkins,against,55.6,50-60,True
149,2023-10-24,Morecambe,J. Turnbull,72,Own Goal,Morecambe,0-1,L,Nigel Adkins,against,50.8,50-60,True
150,2023-10-28,Stockport County,I. Olaofe,5,Goal,Stockport County,0-2,L,Nigel Adkins,against,28.9,20-30,False
155,2023-12-09,Newport County,W. Evans,22,Goal,Newport County,2-1,W,Nigel Adkins,against,59.3,50-60,True
156,2023-12-16,Walsall,J. Matt,64,Goal,Walsall,0-1,L,Nigel Adkins,against,47.2,40-50,False
157,2023-12-23,Swindon Town,D. Kemp,22,Goal,Swindon Town,2-1,W,Nigel Adkins,against,51.9,50-60,True
161,2024-01-06,Barrow,D. Campbell,80,Goal,Barrow,0-1,L,Nigel Adkins,against,50.1,50-60,True
162,2024-01-13,Milton Keynes Dons,A. Gilbey,9,Goal,Milton Keynes Dons,1-2,L,Nigel Adkins,against,35.5,30-40,False
164,2024-02-10,Doncaster Rovers,J. Ironside,15,Goal,Doncaster Rovers,1-2,L,Nigel Adkins,against,53.7,50-60,True
